In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics

In [2]:
# Input Parameters

n        = 1024
b_size   = 1
n_epochs = 50

In [3]:
def createPointCloud(n:int, k:int, k_eff:int, vola:float=0.3, seedVal:int=5493) -> pd.DataFrame:
    '''
    creates dataset
    '''

    assert k>=k_eff, "Dimension `k` must be bigger than effective dimension `k_eff`."
    assert vola >= 0.0, "Error vola must be non-negative."

    # set seed
    np.random.seed(seedVal)
    
    # get data
    X         = np.random.normal(0, 1, n*k).reshape(n, -1)
    id_eff    = np.random.choice(a=range(X.shape[1]), size=k_eff, replace=False)

    # beta
    beta      = np.random.normal(0, 4, X.shape[1]).round(3)
    beta[[i for i in range(len(beta)) if i not in id_eff]] = 0.
    
    # debug
    beta     = np.zeros_like(beta)
    beta[1] = -2
    beta[3] = 1.5
    beta[7] = 4

    # response : `y`
    y = X@beta 
    y += np.random.normal(loc=0, scale=vola, size=y.shape)
    
    outDict = {'y' : y}
    outDict.update( {f'x_{i}' : x for i, x in enumerate(X.T, start=1)})
    
    return pd.DataFrame(outDict)

In [4]:
class pointCloud(Dataset):
    
    def __init__(self, df:pd.DataFrame):
        self.dataframe = df
        self.DF = df.to_numpy()
        
    def __len__(self):
        return len(self.dataframe)
        
    def __getitem__(self, idx):
        assert idx in range(self.__len__())
        return self.DF[idx,:]

In [5]:
# Create data

# sample data
k, k_eff = 15, 5
df = createPointCloud(n, k, k_eff)

# ...Dataset
dset_train = pointCloud(df.iloc[:round(0.8*len(df))])
dset_test  = pointCloud(df.iloc[round(0.8*len(df)):])

# ...Dataloader
trainloader = DataLoader(dataset=dset_train, batch_size = b_size, num_workers=1, pin_memory=True)
testloader  = DataLoader(dataset=dset_test,  batch_size = len(dset_test), num_workers=1, pin_memory=True)

In [6]:
# Model
class LinMod(nn.Module):
    
    def __init__(self,):
        super().__init__()
        self.flatten = nn.Flatten()
        self.lin     = nn.Linear(in_features=k, out_features=1, bias=True)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.lin(x)
        
        return x[:,0]
    
    def eval(self,):
        self.coef_ = list(self.parameters())[0].data.numpy()[0]
        


# create instance 
lm1 = LinMod()

In [7]:
# optimizer, criterion
criterion = nn.MSELoss() # L1Loss
optimizer = optim.SGD(params=lm1.parameters(), lr=0.001, weight_decay=0.000001)

In [8]:
running_loss = 0.0

# epoch loop
for i in range(n_epochs):
    print('Epoch : ', i+1)
    # batch loop
    for j,b in enumerate(trainloader):
        y, x = b[:,0].type(torch.float), b[:,1:].type(torch.float)
        
        # backprop
        optimizer.zero_grad()
        
        y_hat = lm1(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if j % 512 == 511:    # print every 2000 mini-batches
            print(f'[{i + 1}, {i + 1:2d}] loss: {running_loss / 5:.4f}')
            running_loss = 0.0
        
print('Training: Done.')

with torch.no_grad():
    for b in testloader:
        y, x = b[:,0].type(torch.float), b[:,1:].type(torch.float)
        y_hat_test = lm1(x)
        y_test = y
        
        break
        

Epoch :  1
[1,  1] loss: 1018.8865
Epoch :  2
[2,  2] loss: 136.4118
Epoch :  3
[3,  3] loss: 20.2651
Epoch :  4
[4,  4] loss: 15.1885
Epoch :  5
[5,  5] loss: 14.8837
Epoch :  6
[6,  6] loss: 14.8668
Epoch :  7
[7,  7] loss: 14.8672
Epoch :  8
[8,  8] loss: 14.8678
Epoch :  9
[9,  9] loss: 14.8680
Epoch :  10
[10, 10] loss: 14.8681
Epoch :  11
[11, 11] loss: 14.8681
Epoch :  12
[12, 12] loss: 14.8681
Epoch :  13
[13, 13] loss: 14.8681
Epoch :  14
[14, 14] loss: 14.8681
Epoch :  15
[15, 15] loss: 14.8681
Epoch :  16
[16, 16] loss: 14.8681
Epoch :  17
[17, 17] loss: 14.8681
Epoch :  18
[18, 18] loss: 14.8681
Epoch :  19
[19, 19] loss: 14.8681
Epoch :  20
[20, 20] loss: 14.8681
Epoch :  21
[21, 21] loss: 14.8681
Epoch :  22
[22, 22] loss: 14.8681
Epoch :  23
[23, 23] loss: 14.8681
Epoch :  24
[24, 24] loss: 14.8681
Epoch :  25
[25, 25] loss: 14.8681
Epoch :  26
[26, 26] loss: 14.8681
Epoch :  27
[27, 27] loss: 14.8681
Epoch :  28
[28, 28] loss: 14.8681
Epoch :  29
[29, 29] loss: 14.8681


In [9]:
# compute test accuracy
mse = torchmetrics.MeanSquaredError()
mse(y_test, y_hat_test.flatten())

tensor(0.0797)

In [11]:
0.3*np.sqrt(1024)/15.

0.64

In [ ]:
# visually assess parameter estimates
lm1.eval()
lm1.coef_.round(2)